In [ ]:
#@title Copyright 2020 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=jBx3md1PDkJqw3JBk_gZyDS_V9dMhvIMovboDtJRStE&tc=yinFJDFp92Q-MgF6Pw5saOA7TW59JK9QQGisM5rmglU&cc=xgAVaweUdUjbkdQSAP2PmxewyYR0OScW9fpUIzVOlCM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJrZ1JrgfGdolJWKU-oWEl1FN16uICo-kf8LHm3aXaFx-L_tLhM4R08

Successfully saved authorization token.


In [19]:
# Import the Landsat 7 Surface Temperature.
lst = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
#lst=ee.ImageCollection('MODIS/061/MOD11A1')
#lst=ee.ImageCollection('MODIS/061/MYD11A1')
# Final date of interest (exclusive).
i_date = '2021-10-31'
#i_date='2000-02-24'
#i_date='2002-07-04'
#f_date='2022-10-09'
#f_date='2022-10-07'
f_date='2022-10-10'
banda='ST_B10'
#banda='LST_Day_1km'
# Define the urban location of interest as a point near Lyon, France.
u_lon = -69.8906
u_lat = -33.5894
u_poi = ee.Geometry.Point(u_lon,u_lat)
scale=30

In [20]:
def sample_point(point, image):
    return image \
        .select([banda]) \
        .sample(
            region=point.geometry(),
            scale=30, 
            numPixels=1
        ) \
        .first() \
        .set('year', image.date().get('year')) \
        .set('point', point.geometry().coordinates())

def sample_image(image):
    return pts.map(lambda point: sample_point(point, image))

# define mapping function
def point_mean(img):
    mean = img.reduceRegion(reducer=ee.Reducer.mean(),
                            geometry=u_poi,
                            scale=scale).get(banda)
    return img.set('date', img.date().format()).set('mean',mean)

In [21]:
import pandas as pd
collection = lst.filterBounds(u_poi)
collectionF = collection.select(banda).filterDate(i_date,f_date)
poi_reduced_imgs = collectionF.map(point_mean)
nested_list = poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2),
                                             ['date','mean']).values().get(0)
df = pd.DataFrame(nested_list.getInfo(), columns=['date','mean'])
df['mean']=0.00341802*df['mean']+149.0-273.15
#df['mean']=0.02*df['mean']-273.15
import os
df.to_csv(os.path.join('.','LST_Landsat9_2021_2022.csv'))